In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# Cargar los datos
test = pd.read_csv(r'C:\Users\de969\OneDrive\Escritorio\proyecto, machine learnig\_marketing\data\test.csv',sep='\t')


In [2]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index,ID             448 non-null    object 
 1   Year_Birth           448 non-null    int64  
 2   Education            448 non-null    object 
 3   Marital_Status       448 non-null    object 
 4   Income               442 non-null    float64
 5   Kidhome              448 non-null    int64  
 6   Teenhome             448 non-null    int64  
 7   Dt_Customer          448 non-null    object 
 8   Recency              448 non-null    int64  
 9   MntWines             448 non-null    int64  
 10  MntFruits            448 non-null    int64  
 11  MntMeatProducts      448 non-null    int64  
 12  MntFishProducts      448 non-null    int64  
 13  MntSweetProducts     448 non-null    int64  
 14  MntGoldProds         448 non-null    int64  
 15  NumDealsPurchases    448 non-null    int

In [4]:
median_income = test['Income'].median()
test['Income'].fillna(median_income, inplace=True)

In [10]:


# Seleccionar las características y el objetivo
features = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines',
            'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
            'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
            'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
            'Z_CostContact', 'Z_Revenue']  # Variables predictoras
target = 'Response'  # Variable objetivo

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(test[features], test[target], test_size=0.2, random_state=42)

# Aplicar resampling al conjunto de entrenamiento
resampler = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

# Crear el pipeline con escalado de características y modelo de clasificación
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', None)
])

# Definir los parámetros del grid search para cada modelo
param_grid = [
    {
        'classifier': [LogisticRegression(solver='liblinear')],
        'classifier__C': [0.1, 1, 10]
    },
    {
        'classifier': [DecisionTreeClassifier()],
        'classifier__max_depth': [10, 10, 15]
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200]
    },
    {
        'classifier': [SVC()],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    },
    {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [3, 5, 7]
    }
]

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_resampled, y_train_resampled)

# Obtener los resultados de la búsqueda de hiperparámetros
results = grid_search.cv_results_
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Imprimir los resultados
print("Resultados de la búsqueda de hiperparámetros:")
print("Mejor modelo:", best_model)
print("Mejores parámetros:", best_params)
print("Mejor puntuación:", best_score)

# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("Métricas de evaluación en el conjunto de prueba:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("ROC AUC:", roc_auc)

Resultados de la búsqueda de hiperparámetros:
Mejor modelo: Pipeline(steps=[('scaler', StandardScaler()), ('classifier', SVC(C=10))])
Mejores parámetros: {'classifier': SVC(C=10), 'classifier__C': 10, 'classifier__kernel': 'rbf'}
Mejor puntuación: 0.8485815602836879
Métricas de evaluación en el conjunto de prueba:
Accuracy: 0.7666666666666667
Precision: 0.21052631578947367
Recall: 0.4
F1-score: 0.27586206896551724
ROC AUC: 0.60625


In [7]:
import pickle

# Guardar el mejor modelo en un archivo pickle
with open('mejor_modelo.pkl', 'wb') as file:
    pickle.dump(best_model, file)


In [1]:


# # Crear el pipeline con escalado de características, técnica de resampling y modelo de clasificación
# pipeline_smote_tomek = Pipeline([
#     ('scaler', StandardScaler()),
#     ('resampler', SMOTE(sampling_strategy='auto')),
#     ('undersampler', TomekLinks(sampling_strategy='majority')),
#     ('classifier', RandomForestClassifier())
# ])

# # Definir los parámetros del grid search
# param_grid_smote_tomek = {
#     'classifier__n_estimators': [50, 100, 200]
# }

# # Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
# grid_search_smote_tomek = GridSearchCV(pipeline_smote_tomek, param_grid_smote_tomek, cv=5, scoring='accuracy')
# grid_search_smote_tomek.fit(X_train, y_train)

# # Obtener los resultados de la búsqueda de hiperparámetros
# best_model_smote_tomek = grid_search_smote_tomek.best_estimator_
# best_params_smote_tomek = grid_search_smote_tomek.best_params_
# best_score_smote_tomek = grid_search_smote_tomek.best_score_

# # Imprimir los resultados
# print("Resultados de la búsqueda de hiperparámetros para SMOTE + TomekLinks:")
# print("Mejor modelo:", best_model_smote_tomek)
# print("Mejores parámetros:", best_params_smote_tomek)
# print("Mejor puntuación:", best_score_smote_tomek)

In [30]:
print(X_test['Income'].dtype)


float64


In [34]:
X_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index,ID             448 non-null    object 
 1   Year_Birth           448 non-null    int64  
 2   Education            448 non-null    object 
 3   Marital_Status       448 non-null    object 
 4   Income               448 non-null    float64
 5   Kidhome              448 non-null    int64  
 6   Teenhome             448 non-null    int64  
 7   Dt_Customer          448 non-null    object 
 8   Recency              448 non-null    int64  
 9   MntWines             448 non-null    int64  
 10  MntFruits            448 non-null    int64  
 11  MntMeatProducts      448 non-null    int64  
 12  MntFishProducts      448 non-null    int64  
 13  MntSweetProducts     448 non-null    int64  
 14  MntGoldProds         448 non-null    int64  
 15  NumDealsPurchases    448 non-null    int

In [9]:
import pickle
from pathlib import Path

def read_pickle(file_path: str):
    """
    Read data from a Pickle file.

    Args:
        file_path (str): Path of the Pickle file.

    Returns:
        object: Data loaded from the Pickle file, or None if there is an error.
    """
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        print("Pickle file read: OK")
        return data
    except (FileNotFoundError, IOError, pickle.PickleError) as err:
        print(f"Failed to read Pickle file {file_path}: {err}")
        return None

# Leer el modelo desde el archivo Pickle
modelo = read_pickle("../models/mejor_modelo.pkl")
# median_income = test['Income'].median()
# test['Income'].fillna(median_income, inplace=True)
# Utilizar el modelo cargado para hacer predicciones, por ejemplo:
# X_test = test  # Tus datos de prueba
# y_pred = modelo.predict(X_test)

Pickle file read: OK
